In [18]:
# chapter 3

import tensorflow as tf

g1 = tf.Graph()
with g1.as_default():
#     在计算图g1中定义变量"v"，并设置初始值为0
    v = tf.get_variable("v", initializer=tf.zeros_initializer, shape=[1])

g2 = tf.Graph()
with g2.as_default():
#     在计算图g2中定义变量"v"，并设置初始值为1
    v = tf.get_variable("v", initializer=tf.ones_initializer, shape=[1])

# 在计算图g1中读取变量"v"的取值
with tf.Session(graph=g1) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
#         在计算图g1中，变量"v"的取值应该为0，所以下面这行会输出[0.]
        print(sess.run(tf.get_variable("v")))

with tf.Session(graph=g2) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
#         在计算图g2中，变量"v"的取值应该为1，所以下面这行输出[1.]
        print(sess.run(tf.get_variable("v")))

[0.]
[1.]


In [30]:
import tensorflow as tf
a = tf.constant([1.0, 2.0], name = 'a')
b = tf.constant([2.0, 3.0], name = 'b')

result0 = a + b

# print(result)
with tf.Session() as sess:
    print(result0.eval())
    tf.global_variables_initializer().run()
    resu = sess.run(result0)
    print(resu)
    
result1 = tf.add(a, b, name="add")

print(result1)

[3. 5.]
[3. 5.]
Tensor("add_18:0", shape=(2,), dtype=float32)


In [4]:
# 实现前向传播算法
import tensorflow as tf

# 声明w1、w2两个变量，这里还通过seed参数设定了随机种子
# 这样可以保证每次运行得到的结果是一样的

w1 = tf.Variable(tf.random_normal([2, 3], stddev = 1, seed = 1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev = 1, seed = 1))

# 暂时将输入的特征向量定义为一个常量。注意这里x是一个1*2的矩阵
x = tf.constant([[0.7, 0.9]])

# 获得前向传播算法神经网络的输出
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
#     sess.run(w1.initializer)
#     sess.run(w2.initializer)
    print(sess.run(y))
    

[[3.957578]]


In [18]:
import tensorflow as tf

w1 = tf.Variable(tf.random_normal([2, 3], stddev = 1, seed = 1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev = 1, seed = 1))

# 定义placeholder作为存放输入数据的地方。这里维度也不一定要定义
# 但如果维度是确定的，那么给出维度可以降低出错的概率

x = tf.placeholder(tf.float32, shape=(1, 2), name = "input")
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(y, feed_dict={x: [[0.7, 0.9]]}))


[[3.957578]]


In [21]:
import tensorflow as tf

w1 = tf.Variable(tf.random_normal([2, 3], stddev = 1, seed = 1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev = 1, seed = 1))

x = tf.placeholder(tf.float32, shape = (3, 2), name = "input")
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(y, feed_dict={x: [[0.7, 0.9], [0.1, 0.4], [0.5, 0.8]]}))

[[3.957578 ]
 [1.1537654]
 [3.1674924]]


In [9]:
# 训练神经网络解决二分类问题
import tensorflow as tf

from numpy.random import RandomState

# 定义训练数据的batch的大小，分批处理，批处理的大小
batch_size = 8

# 定义神经网络的参数
w1 = tf.Variable(tf.random_normal([2, 3], stddev = 1, seed = 1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev = 1, seed = 1))

#在shape的一个维度上使用None可以方便使用不大的batch大小。在训练时需要把数据分成比较小的batch
#但是在测试时，可以一次性使用全部的数据。当数据集比较小时这样比较方便测试，但数据集比较大时，将大量数据放入一个batch可能会导致内存溢出
x = tf.placeholder(tf.float32, shape = (None, 2), name = "input")
y_ = tf.placeholder(tf.float32, shape = (None, 1), name = "input")

# 定义神经网络前向传播的过程
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

# 定义损失函数和反向传播的算法
cross_entroy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0)))
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entroy)

# 通过随机数生成一个模拟数据集
rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size, 2)

# 定义规则来给出样本的标签。在这里所有x1+x2<1的样例都被认为是正样本（比如零件合格）
# 而其他为负样本（比如零件不合格）。和TensorFlow游乐场中的表示法不大一样的地方是在这里
# 使用0来表示负样本，1表示正样本。大部分解决分类问题的神经网络都会采用0和1的表示方法

Y = [[int(x1 + x2 < 1)] for (x1, x2) in X] 

# 创建一个会话来运行TensorFlow程序
with tf.Session() as sess:
#     初始化变量
    tf.global_variables_initializer().run()
    print(sess.run(w1))
    print(sess.run(w2))
    
    '''
    在训练之前神经网络参数的值：
    w1 = [[-0.81131822, 1.48459876, 0.06532937]
          [-2.44270396, 0.0992484, 0.59122431]]
    w2 = [[-0.811318221], [1.48459876], [0.06532937]]
    '''
    
#     设定训练的轮数
    STEPS = 5000
    for i in range(STEPS):
#         每次选取batch_size个样本进行训练
        start = (i * batch_size) % dataset_size
        end = min(start + batch_size, dataset_size)
        
#         通过选取的样本训练神经网络并更新参数
        sess.run(train_step, feed_dict={x: X[start:end], y_: Y[start:end]})
        if i % 1000 == 0:
#             每隔一段时间计算在所有数据上的交叉熵并输出
            total_cross_entropy = sess.run(cross_entroy, feed_dict={x: X, y_:Y})
            print("After %d training step(s), cross entropy on all data is %g" %(i, total_cross_entropy))
        """
        通过这个结果可以发现随着训练的进行，交叉熵是逐渐变小的。交叉熵越小说明预测的结果和真是的结果差距越小
        """
    print(sess.run(w1))
    print(sess.run(w2))
    """
     可以发现这两个参数的取值已经发生了变化，这个变化就是训练的结果。
    它使得这个神经网络能更好的拟合提供的训练数据。
    """
    


[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
[[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]
After 0 training step(s), cross entropy on all data is 0.0674925
After 1000 training step(s), cross entropy on all data is 0.0163385
After 2000 training step(s), cross entropy on all data is 0.00907547
After 3000 training step(s), cross entropy on all data is 0.00714436
After 4000 training step(s), cross entropy on all data is 0.00578471
[[-1.9618275  2.582354   1.6820377]
 [-3.4681718  1.0698231  2.11789  ]]
[[-1.824715 ]
 [ 2.6854665]
 [ 1.418195 ]]
